In [3]:
import mindspore.nn as nn
from mindspore import Tensor
import mindspore.ops.operations as P

def channel_shuffle(x, groups):
    batchsize, num_channels, height, width = x.shape
    channels_per_group = num_channels // groups
    # reshape
    x = P.Reshape(x,(batchsize, groups, channels_per_group, height, width))
    x = P.Transpose()(x, (1, 2))
    x = P.Reshape()(x, (batchsize, -1, height, width))
    return x

class ShuffleV2Block(nn.Cell):
    def __init__(self, inp, oup, stride, activation="ReLU"):
        super(ShuffleV2Block, self).__init__()
        if not (1 <= stride <= 3):
            raise ValueError("illegal stride value")
        self.stride = stride
        branch_features = oup // 2


        if self.stride > 1:
            self.branch1 = nn.SequentialCell([
                self.depthwise_conv(inp, inp, kernel_size=3, stride=self.stride, padding=1),
                nn.BatchNorm2d(inp),
                nn.Conv2d(inp,branch_features,kernel_size=1,stride=1,pad_mode=0,has_bias=False),
                nn.BatchNorm2d(branch_features),
                nn.ReLU(),
            ])
        else:
            self.branch1 = nn.SequentialCell()

        self.branch2 = nn.SequentialCell([
            nn.Conv2d(
                inp if (self.stride > 1) else branch_features,
                branch_features,
                kernel_size=1,
                stride=1,
                padding=0,
                has_bias=False,
            ),
            nn.BatchNorm2d(branch_features),
            nn.ReLU(),
            self.depthwise_conv(
                branch_features,
                branch_features,
                kernel_size=3,
                stride=self.stride,
                padding=1,
            ),
            nn.BatchNorm2d(branch_features),
            nn.Conv2d(
                branch_features,
                branch_features,
                kernel_size=1,
                stride=1,
                padding=0,
                has_bias=False,
            ),
            nn.BatchNorm2d(branch_features),
            nn.ReLU(),
        ])

    @staticmethod
    def depthwise_conv(i, o, kernel_size, stride=1, padding=0, bias=False):
        return nn.Conv2d(i, o, kernel_size, stride, padding, has_bias=bias, group=i, pad_mode="pad")

    def construct(self, x):
        if self.stride == 1:
            x1, x2 = P.Split(axis=1,output_num=2)(x)
            out = P.Concat(axis=1)((x1, self.branch2(x2)))
        else:
            out = P.Concat(axis=1)((self.branch1(x), self.branch2(x)))
        out = channel_shuffle(out, 2)
        return out

class shuffleNetV2(nn.Cell):
    def __init__(self, model_size="1.0x", out_stages=(2, 3, 4),  kernal_size=3, with_last_conv=False, activation="ReLU"):
        super(shuffleNetV2, self).__init__()
        self.stage_repeats = [4, 8, 4]
        self.model_size = model_size
        self.out_stages = out_stages
        self.with_last_conv = with_last_conv
        self.kernal_size = kernal_size
        self.activation = activation
        if model_size == "0.5x":
            self._stage_out_channels = [24, 48, 96, 192, 1024]
        elif model_size == "1.0x":
            self._stage_out_channels = [24, 116, 232, 464, 1024]
        elif model_size == "1.5x":
            self._stage_out_channels = [24, 176, 352, 704, 1024]
        elif model_size == "2.0x":
            self._stage_out_channels = [24, 244, 488, 976, 2048]
        else:
            raise NotImplementedError

        input_channels = 3
        output_channels = self._stage_out_channels[0]
        self.conv1 = nn.SequentialCell([
            nn.Conv2d(input_channels, output_channels, kernel_size=3, stride=2, padding=1, pad_mode="pad", has_bias=False),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(),
        ])
        input_channels = output_channels
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode="same")
        stage_names = ["stage{}".format(i) for i in [2, 3, 4]]
        for name, repeats, output_channels in zip(
            stage_names, self.stage_repeats, self._stage_out_channels[1:]
        ):
            seq = [
                ShuffleV2Block(
                    input_channels, output_channels, 2, activation=activation
                )
            ]
            for i in range(repeats - 1):
                seq.append(
                    ShuffleV2Block(
                        output_channels, output_channels, 1, activation=activation
                    )
                )
            setattr(self, name, nn.SequentialCell(*seq))
            input_channels = output_channels
        output_channels = self._stage_out_channels[-1]
        if self.with_last_conv:
            conv5 = nn.SequentialCell(
                nn.Conv2d(input_channels, output_channels, kernel_size=1,stride=1,padding=0,has_bias=False),
                nn.BatchNorm2d(output_channels),
                nn.ReLU,
            )
            self.stage4.add_module("conv5", conv5)

    def construct(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        output = []
        for i in range(2, 5):
            stage = getattr(self, "stage{}".format(i))
            x = stage(x)
            if i in self.out_stages:
                output.append(x)
        return tuple(output)

In [4]:
conv = nn.Conv2d(3,3)
bn = nn.BatchNorm2d(3)

TypeError: missing a required argument: 'kernel_size'

In [ ]:
import mindspore
from mindspore.common.initializer import initializer, XavierUniform
import numpy as np
tensor1 = initializer(XavierUniform(), [2, 3], mindspore.float32)
tensor2 = initializer('xavier_uniform', [1, 2, 3], mindspore.float32)
x = mindspore.Tensor(np.random.randint(0, 255, (1, 3, 320, 320)), mindspore.float32)

In [ ]:
import mindspore.nn as nn
up = nn.ResizeBilinear()(x, scale_factor=0.5)

In [3]:
conv = nn.Conv2d(3,64,3,1)

In [6]:
conv.output_padding

AttributeError: The 'Conv2d' object has no attribute 'output_padding'.

In [9]:
celllist = nn.CellList(nn.Conv2d(3,16,3),nn.BatchNorm2d(16),nn.ReLU())

In [10]:
celllist[0]

IndexError: For 'CellList', the value of 'index' should be a number in range [0, 0), but got 0.

In [10]:
import numpy as np
import mindspore
x = Tensor(np.ones(shape=[10,4,8]), mindspore.float32)
y = Tensor(np.ones(shape=[8]),mindspore.float32)
batmatmul = P.BatchMatMul()
output = batmatmul(x, y)

ValueError: For 'BatchMatMul', input 'x' should be greater than or equal to 3, input 'y' should be greater than or equal to 2, but got 'x' size: 3, 'y' size: 1.

In [13]:
a = Tensor(np.ones(shape=[40, 8]), mindspore.float32)
b = Tensor(np.ones(shape=[1, 8]), mindspore.float32)
matmul = P.MatMul(transpose_b=True)
output = matmul(a, b)

In [14]:
output

Tensor(shape=[40, 1], dtype=Float32, value=
[[8.00000000e+000],
 [8.00000000e+000],
 [8.00000000e+000],
 ...
 [8.00000000e+000],
 [8.00000000e+000],
 [8.00000000e+000]])